In [20]:
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt 
import pandas as pd 
#import backtesting as bt
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

GOOG.tail()

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def BOLL(values, n, m, component):
    """
    Return Bollinger Bands for `values`, at
    each step taking into account `n` previous values
    and `m` standard deviations away from the mean.
    """
    sma = SMA(values, n)
    std = pd.Series(values).rolling(n).std()
    three = (sma - m * std, sma, sma + m * std)
    return three[component]

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    s1 = 1
    s2 = 2
    s3 = 3
    
    def init(self):
        # Precompute the two moving averages
        # self.boll_1 = self.I(BOLL, self.data.Close, 20, self.s1)
        # self.boll_2 = self.I(BOLL, self.data.Close, 20, self.s2)
        # self.boll_3 = self.I(BOLL, self.data.Close, 20, self.s3)
        self.boll_2_up = self.I(BOLL, self.data.Close, 20, self.s2, 2)
        self.boll_2_down = self.I(BOLL, self.data.Close, 20, self.s2, 0)
    
    def next(self):
       
        #if self.data.Close[-1] < self.boll_2_down[-1]:
        
        #If price crossover the lower band of boll_2, buy
        if self.data.Close[-1] < self.boll_2_down[-1]:
            self.position.close()
            self.buy()
    
        # If price is between the upper band of boll_2 and boll_3, AND the price continue to drop for three days, sell
        elif self.data.Close[-1] > self.boll_2_up[-1]:
            self.position.close()
            self.sell()
            
        
bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.022346
Equity Final [$]                   1531.60586
Equity Peak [$]                       10000.0
Return [%]                         -84.683941
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                  -19.757819
Volatility (Ann.) [%]                26.06507
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -92.215765
Avg. Drawdown [%]                  -92.215765
Max. Drawdown Duration     3087 days 00:00:00
Avg. Drawdown Duration     3087 days 00:00:00
# Trades                                  246
Win Rate [%]                        32.520325
Best Trade [%]                       40.11541
Worst Trade [%]                    -47.528175
Avg. Trade [%]                    

In [ ]:
import backtesting as bkt 
bt.plot()

ValueError: failed to validate DatetimeTickFormatter(id='p1138', ...).days: expected a value of type str, got ['%d %b', '%a %d'] of type list